In [1]:
from google.colab import drive
drive.mount('/content/drive')
NUM_TRIALS = 20

Mounted at /content/drive


In [2]:
pip install gspread==3.6


  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [3]:
import numpy as np
import pdb
import pandas as pd
import numbers
from google.colab import auth
import os
auth.authenticate_user()

import gspread 
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1F8A5n3WVr9WXHDXZhv2w0BPaMC8uymUyxEWdI9UX6xg/edit#gid=0')

In [4]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [5]:
# !sh setup.sh

In [6]:
# %cd drive/My Drive/mixout/

In [7]:
!git clone https://github.com/leedtan/ModernML_TinyBert.git

Cloning into 'ModernML_TinyBert'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 599 (delta 112), reused 102 (delta 58), pack-reused 443
Receiving objects: 100% (599/599), 22.17 MiB | 9.77 MiB/s, done.
Resolving deltas: 100% (354/354), done.


In [8]:
os.chdir('ModernML_TinyBert')
# !python download_glue_data.py

In [9]:
os.chdir('revisit-bert-finetuning')

In [10]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 13.6MB/s 
     |████████████████████████████████| 1.1MB 31.2MB/s 
     |████████████████████████████████| 3.7MB 48.4MB/s 
     |████████████████████████████████| 133kB 52.3MB/s 
     |████████████████████████████████| 890kB 51.6MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 7.2MB 57.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e6ad935a93ecdb9a3ea089fbe8479752573cb6d0f4286244889d7f27768a836a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.47 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [11]:
!git checkout some_frozen_mixout_reg_scaling

Branch 'some_frozen_mixout_reg_scaling' set up to track remote branch 'some_frozen_mixout_reg_scaling' from 'origin'.
Switched to a new branch 'some_frozen_mixout_reg_scaling'


In [12]:
!cat ryan.sh

../../revisit-bert-finetuning/bert/bin/python run_glue_datasets.py \
    --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
    --do_train --data_dir ../../revisit-bert-finetuning/glue_data --max_seq_length 64 \
    --per_gpu_eval_batch_size 8 --weight_decay 0 --seed 1 \
    --overwrite_output_dir --do_lower_case --per_gpu_train_batch_size 8 \
    --gradient_accumulation_steps 4 --logging_steps 0 --num_loggings 10 \
    --save_steps 0 --test_val_split --use_torch_adamw --cache_dir /home/ubuntu/hf-transformers-cache \
    --num_train_epochs 3.0 --warmup_ratio 0.1 --learning_rate 2e-05 \
    --output_dir tests/FULLTESTS/decay_50_to_20 \
    --reinit_pooler --normalize --mixout_layers 12 --mixout .2 \
    --trials 20 --mixout_decay 0.5

# ../../revisit-bert-finetuning/bert/bin/python run_glue_datasets.py \
#     --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
#     --do_train --data_dir ../../revisit-bert-finetuning/glue_data --max_seq_

In [13]:
!git pull

Already up to date.


In [14]:

sheet = wb.worksheet('results')
def get_df(sheet):
  df = pd.DataFrame(sheet.get_all_values())
  df.columns = df.iloc[0,:]
  df.index = df.iloc[:,0]
  df = df.iloc[1:,1:]
  return df
  
def get_row_num(df, sheet, paramnames):
  if paramnames in df.index:
    rowidx = df.index.to_list().index(paramnames) + 1
  else:
    current_max_row = len(sheet.get_all_values())
    rowidx = int(current_max_row) + 1
  return rowidx
def check_run(paramnames, task, df = None, sheet = None):
  if 1:# df is None:
    df = get_df(sheet)
  if paramnames in df.index:
    idx = df.columns.to_list().index(task)
    row = df.loc[paramnames,:]
    rowiloc = row.iloc[idx+1]
    if rowiloc == '':
      return True
    if not isinstance(rowiloc, numbers.Number):
      if isinstance(rowiloc, str) and rowiloc.isnumeric():
        rowiloc = int(rowiloc)
      else:
        return False
    if rowiloc >= NUM_TRIALS:
      return False
  return True
def run(paramnames, task, df, sheet, params = {}):
  df = get_df(sheet)
  if paramnames not in df.index:
    row_idx = get_row_num(df, sheet, paramnames)
    sheet.update(f"A{row_idx}", [[paramnames]])
    df = get_df(sheet)
  row_idx = get_row_num(df, sheet, paramnames)
  df = get_df(sheet)
  idx = df.columns.to_list().index(task)
  count = df.iloc[row_idx-1, idx+1]
  if count == '':
    count = 0
    avg = 0
  avg = df.iloc[row_idx-1, idx]
  if avg == '':
    avg = 0
  avg = float(avg)
  count = int(count)
  
  all_datasets = ['rte','sts-b','mrpc','cola']
  metrics = ['test_acc', 'test_pearson', 'test_acc', 'test_mcc']
  metrics = dict(zip(all_datasets, metrics))
  result_key = metrics[args.task_name.lower()]

  args.seed = seed = count + 1
  args.output_dir = (
      output_dir + "_DATASET_" + args.task_name.lower() + "_SEED_" + str(seed)
  )
  results = run_glue_main(args)
  print(type(results))
  print(results)
  if 'acc' in results:
    score = results['acc']
  elif 'pearson' in results:
    score = results['pearson']
  print('avg',type(avg),avg,'count',type(count),count,'score',type(score),score,)
  newavg = (avg * count + score) / (count + 1)
  sheet.update(f"{chr(ord('a') + idx + 2)}{row_idx+1}", [[count+1]])
  sheet.update(f"{chr(ord('a') + idx + 1)}{row_idx+1}", [[newavg]])

  return True
def simulate(paramnames, tasks, df = None, sheet = None, params = {}):
  if df is None:
    df = get_df(sheet)
  for task in tasks:
    i = 0
    while check_run(paramnames, task, df, sheet) and i < NUM_TRIALS:
      i += 1
      run(paramnames, task, df, sheet, params = params)
      df = get_df(sheet)

In [15]:
def dict2obj(d):
    if isinstance(d, list):
        d = [dict2obj(x) for x in d]
    if not isinstance(d, dict):
        return d
    class C(object):
        def __init__(self):
          pass
        def __call__(self):
          pass
        pass
    o = C()
    for k in d:
        o.__dict__[k] = dict2obj(d[k])
    return o
dict2obj({'b':2}).b

2

In [16]:
args ={'model_type': 'bert', 'model_name_or_path': 'bert-large-uncased', 'task_name': 'RTE', 
    'do_train':True, 'data_dir': "/content/drive/My Drive/mixout/ModernML_TinyBert/glue_data", 'max_seq_length': 64, 
    'per_gpu_eval_batch_size': 8, 'weight_decay': 0, 'seed': 1, 
    'overwrite_output_dir':True, 'do_lower_case':True, 'per_gpu_train_batch_size': 8, 
    'gradient_accumulation_steps': 4, 'logging_steps': 0, 'num_loggings': 10, 
    'save_steps': 0, 'test_val_split':True, 'use_torch_adamw':True, 
    'cache_dir': "/content/drive/My Drive/mixout/ModernML_TinyBert/hf-transformers-cache" ,
    'num_train_epochs': 3.0, 'warmup_ratio': 0.1, 'learning_rate': 2e-05 ,
    'output_dir': 'tests/FULLTESTS/classic', 'all_datasets':True, 
    'reinit_pooler': True, 'normalize': True, 'mixout_layers': 12, 'mixout': .3, 
    'trials': 10}

for name, default_val in zip(["data_dir", "model_type", "model_name_or_path", "task_name", "output_dir",
 "config_name", "tokenizer_name", "cache_dir", "max_seq_length", "do_train","do_lower_case", "save_best",
  "save_last", "train_batch_size", "per_gpu_train_batch_size", "per_gpu_eval_batch_size", "gradient_accumulation_steps",
   "learning_rate", "layerwise_learning_rate_decay", "weight_decay", "adam_epsilon", "max_grad_norm", "num_train_epochs", 
   "max_steps", "warmup_steps", "warmup_ratio", "weight_logging_steps", "logging_steps", "num_loggings", "save_steps", 
   "no_cuda","overwrite_output_dir", "overwrite_cache", "seed", "fp16", "fp16_opt_level", "local_rank", "server_ip",
   "server_port","use_bertadam","use_torch_adamw","downsample_trainset", "resplit_val", "reinit_layers", "mixout_layers",
    "unfreeze_after_epoch", "reinit_pooler","l2_scaling","normalize","all_datasets","layer_mixout","rezero_layers", "mixout",
     "mixout_decay", "trials", "prior_weight_decay", "test_val_split",'frozen_layers', 'finetune_layers',
      'l2_reg_decay', 'l2_reg_mult'], 
      [None, None, None, None, None, '', '', '', 128, False, False, False, False, 0, 8, 8,
1, 5e-5, 1.0, 0.0, 1e-8, 1.0, 3.0, -1, 0, 0, 10, 0, 0, 500, False, False, False, 42,
False, '01', -1, '', '', False, False, -1, 0, 0, 0, 0, False, False, False, False, False, 
0, 0.0, 1.0, NUM_TRIALS, False, False, 0, 0, 1.0, 3e-3]):
    if name not in args:
        args[name] = default_val

In [17]:

args['task_name']

'RTE'

In [18]:

!ls

collab.ipynb				      README.md
emily_run_glue_experiment.sh		      repo_illustration.png
finetuning_torchvision_models_tutorial.ipynb  requirements.txt
glue_utils.py				      run_glue_datasets.py
__init__.py				      run_glue_experiment.py
LICENSE					      run_glue.py
manipulate_model.py			      ryan_run_glue_experiment.sh
mixout-fan-in.ipynb			      ryan.sh
mixout.ipynb				      sample_commands
mixout.py				      summarize_results.py
model_utils.py				      train.py
options.py				      validate.py
prior_wd_optim.py


In [ ]:
from run_glue import main as run_glue_main
from options import get_parser
import os

args =dict2obj(args)
output_dir = args.output_dir
data_dir = args.data_dir

def experiment(seeds):
    for seed in seeds:

        all_datasets = ['rte','sts-b','mrpc','cola']
        metrics = ['test_acc', 'test_pearson', 'test_acc', 'test_mcc']
        metrics = dict(zip(all_datasets, metrics))
        result_key = metrics[args.task_name.lower()]

        args.seed = seed
        args.output_dir = (
            output_dir + "_DATASET_" + args.task_name.lower() + "_SEED_" + str(seed)
        )
        results = run_glue_main(args)
        
        score = results[result_key]

DATASETS = ["RTE", "MRPC", "STS-B"]
# DATASETS = ["RTE"]


# for dataset in DATASETS:
#     seeds = range(args.trials)
#     args.task_name = dataset
#     args.data_dir = os.path.join(data_dir, args.task_name)
#     experiment(seeds)
def run_real(paramnames, tasks, df = None, sheet = None, params = {}):
  if df is None:
    df = get_df(sheet)
  for task in tasks:
    args.task_name = task
    # params.data_dir = os.path.join(data_dir, args.task_name)
    args.data_dir = os.path.join(data_dir, args.task_name)
    i = 0
    while check_run(paramnames, task, df, sheet) and i < NUM_TRIALS:
      i += 1
      run(paramnames, task, df, sheet, params = params)
      df = get_df(sheet)
lnum = [args.frozen_layers, args.mixout_layers, args.finetune_layers, args.mixout_layers]
paramnames = f"lay_{lnum[0]}_{lnum[1]}_{lnum[2]}_{lnum[3]}_reg_{args.l2_reg_mult}_regdecay_{args.l2_reg_decay}_"
run_real(paramnames, DATASETS, df = None, sheet = sheet, params = args)

01/02/2021 13:44:06 - WARNING - run_glue -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
01/02/2021 13:44:07 - INFO - filelock -   Lock 140397426830864 acquired on /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8.lock
01/02/2021 13:44:07 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpv_031oq5


01/02/2021 13:44:07 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json in cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
01/02/2021 13:44:07 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
01/02/2021 13:44:07 - INFO - filelock -   Lock 140397426830864 released on /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8.lock
01/02/2021 13:44:07 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at 

01/02/2021 13:44:09 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /content/drive/My Drive/mixout/ModernML_TinyBert/hf-transformers-cache/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
01/02/2021 13:44:09 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": "rte",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_en


 unfreezing mixout layers 




iteration:  29%|██▉       | 91/312 [04:53<11:46,  3.20s/it]/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "

Epoch: 100%|██████████| 3/3 [50:20<00:00, 1006.72s/it]
01/02/2021 14:35:09 - INFO - run_glue -   Loading features from cached file /content/drive/My Drive/mixout/ModernML_TinyBert/glue_data/RTE/cached_dev_bert-large-uncased_64_rte
01/02/2021 14:35:10 - INFO - run_glue -   Loading features from cached file /content/drive/My Drive/mixout/ModernML_TinyBert/glue_data/RTE/cached_dev_bert-large-uncased_64_rte
01/02/2021 14:35:11 - INFO - run_glue -    global_step = 234, average loss = 0.5717437755571216
01/02/2021 14:35:11 - INFO - run_glue -   Saving model checkpoint to tests/FULLTESTS/classic_DATASET_rte_SEED_5


...


Epoch: 100%|██████████| 3/3 [49:56<00:00, 998.76s/it]
01/02/2021 20:28:00 - INFO - run_glue -   Loading features from cached file /content/drive/My Drive/mixout/ModernML_TinyBert/glue_data/RTE/cached_dev_bert-large-uncased_64_rte
01/02/2021 20:28:02 - INFO - run_glue -   Loading features from cached file /content/drive/My Drive/mixout/ModernML_TinyBert/glue_data/RTE/cached_dev_bert-large-uncased_64_rte
01/02/2021 20:28:03 - INFO - run_glue -    global_step = 234, average loss = 0.5778654813129678
01/02/2021 20:28:03 - INFO - run_glue -   Saving model checkpoint to tests/FULLTESTS/classic_DATASET_rte_SEED_12


<class 'dict'>
{'acc': 0.7266187050359713, 'loss': 0.541858908202913, 'step': 234.0}
avg <class 'float'> 0.6971877045 count <class 'int'> 11 score <class 'numpy.float64'> 0.7266187050359713


01/02/2021 20:28:06 - WARNING - run_glue -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
01/02/2021 20:28:06 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
01/02/2021 20:28:06 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LAB


 unfreezing mixout layers 




iteration:  40%|████      | 125/312 [06:41<09:59,  3.21s/it]

In [ ]:
!ls '/content/drive/My Drive/mixout/ModernML_TinyBert/glue_data'

In [ ]:
3

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!pwd

In [ ]:

args.data_dir

In [ ]:
from run_glue import main as run_glue_main
from options import get_parser
import os

args =dict2obj(args)
output_dir = args.output_dir
data_dir = args.data_dir


DATASETS = ["RTE", "MRPC", "CoLA", "STS-B"]
DATASETS = ["RTE", "MRPC", "STS-B"]


def experiment(seeds):
    for seed in seeds:
        args.seed = seed
        args.output_dir = (
            output_dir + "_DATASET_" + args.task_name.lower() + "_SEED_" + str(seed)
        )
        run_glue_main(args)


if __name__ == "__main__":
    # revisiting finetuned bert (https://arxiv.org/pdf/2006.05987.pdf) uses 20 random seeds
    seeds = range(args.trials)
    if not args.all_datasets:
        args.data_dir = os.path.join(data_dir, args.task_name)
        experiment(seeds)
    else:
        for dataset in DATASETS:
            args.task_name = dataset
            args.data_dir = os.path.join(data_dir, args.task_name)
            experiment(seeds)

In [ ]:
# !echo '''python run_glue_datasets.py \
#     --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
#     --do_train --data_dir /content/ModernML_TinyBert/glue_data/RTE --max_seq_length 16 \
#     --per_gpu_eval_batch_size 64 --weight_decay 0 --seed 0 \
#     --overwrite_output_dir --do_lower_case --per_gpu_train_batch_size 32 \
#     --gradient_accumulation_steps 1 --logging_steps 0 --num_loggings 10 \
#     --save_steps 0 --test_val_split --use_torch_adamw --cache_dir /content/ModernML_TinyBert/hf-transformers-cache \
#     --num_train_epochs 3.0 --warmup_ratio 0.1 --learning_rate 2e-05 \
#     --output_dir bert_output/REINIT5/RTE/SEED0 \
#     --reinit_pooler --reinit_layers 5''' > sample_commands/run.sh

!echo '''python run_glue_datasets.py \
    --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
    --do_train --data_dir "/content/drive/My Drive/mixout/ModernML_TinyBert/glue_data" --max_seq_length 64 \
    --per_gpu_eval_batch_size 8 --weight_decay 0 --seed 1 \
    --overwrite_output_dir --do_lower_case --per_gpu_train_batch_size 8 \
    --gradient_accumulation_steps 4 --logging_steps 0 --num_loggings 10 \
    --save_steps 0 --test_val_split --use_torch_adamw --cache_dir "/content/drive/My Drive/mixout/ModernML_TinyBert/hf-transformers-cache" \
    --num_train_epochs 3.0 --warmup_ratio 0.1 --learning_rate 2e-05 \
    --output_dir tests/FULLTESTS/classic --all_datasets \
    --reinit_pooler --normalize --mixout_layers 12 --mixout .3 \
    --trials 10''' > sample_commands/run.sh

In [ ]:
!git pull

In [ ]:
!sh sample_commands/run.sh